In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Importando os Dados

são dois os conjuntos de dados de interesse

* os dados cadastrais dos filmes.
* os dados dos ratings dos usuários ```u.data```

Da descrição do dataset temos

```
RATINGS FILE DESCRIPTION
================================================================================

All ratings are contained in the file "ratings.dat" and are in the
following format:

UserID::MovieID::Rating::Timestamp

- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings
              
MOVIES FILE DESCRIPTION
================================================================================

Movie information is in the file "movies.dat" and is in the following
format:

MovieID::Title::Genres

- Titles are identical to titles provided by the IMDB (including
year of release)
- Genres are pipe-separated and are selected from the following genres:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western

- Some MovieIDs do not correspond to a movie due to accidental duplicate
entries and/or test entries
- Movies are mostly entered by hand, so errors and inconsistencies may exist```


## Importação da bae de ratings (u.data)

In [68]:
ratingsDF = pd.read_csv("..\\RawData\\ml-1m\ml-1m\\ratings.dat", header=None, sep='::', names=["uid", "id", "rating",'timestamp'])

C:\Users\souza\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [69]:
print("O tamanho do dataset #linhas:", len(ratingsDF))

O tamanho do dataset #linhas: 1000209


preview do dataset de filmes

In [70]:
ratingsDF.head(5)

,uid,id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


conferindo os tipos de dados

In [71]:
ratingsDF.dtypes

uid          int64
id           int64
rating       int64
timestamp    int64
dtype: object

Embora não precisássemos nos importar com a coluna de *timestamp*, isso pode nos ajudar a ganhar insight sobre o tipo de dado que encontraremos na base na etapa de exploração e por hora, prefiro mantê-la com a unidade apropriada

In [72]:
ratingsDF_typed = ratingsDF
ratingsDF_typed['timestamp'] = pd.to_datetime(ratingsDF['timestamp'], unit='s')

In [73]:
ratingsDF_typed['rating'] = ratingsDF_typed['rating'].astype('category')

In [74]:
ratingsDF_typed.head(5)

,uid,id,rating,timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


In [75]:
print("O tamanho do dataset de ratings após ter sido carregado e tipado corretamente é:", ratingsDF_typed.size, "B")

O tamanho do dataset de ratings após ter sido carregado e tipado corretamente é: 4000836 B


## Importação da bae cadastral (u.item)

In [76]:
moviesDFColsNames = "MovieID::Title::Genres".split('::')

In [77]:
moviesDFColsNames = [x.strip().replace(' ','_') for x in moviesDFColsNames]

In [78]:
print(moviesDFColsNames)

['MovieID', 'Title', 'Genres']


In [79]:
moviesDF = pd.read_csv("..\\RawData\\ml-1m\ml-1m\\movies.dat", header=None, sep='::', encoding="ISO-8859-1", names=moviesDFColsNames)

C:\Users\souza\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


Observações sobre a leitura do arquivo

> *Aparentemente o dataset está em codificação ISO.*
> Apesar dos metadados da base terem dito que o arquivo era separado por tab, o dado acabou mostando que a separação era por '|'

In [80]:
moviesDF.head(5)

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [81]:
print("O tamanho do dataset #linhas:", len(moviesDF))

O tamanho do dataset #linhas: 3883


In [82]:
categprocal_columns = ['Genres']
for c in categprocal_columns:
    moviesDF[c] = moviesDF[c].astype('category')

In [83]:
moviesDF_typed = moviesDF

In [84]:
print("O tamanho do dataset de ratings após ter sido carregado e tipado corretamente é:", moviesDF_typed.size, "B")

O tamanho do dataset de ratings após ter sido carregado e tipado corretamente é: 11649 B


## Explorando os dados

Embora para o problema em questão, não estejamos interessados na maioria das infomações do dataset, por exemplo, datas e gêneros de filmes, isso pode nos ajudar a compreender melhor os dados.

In [85]:
ratingsDF_typed.describe(include='all')

,uid,id,rating,timestamp
count,1.000209e+06,1.000209e+06,1000209.0,1000209
unique,NaN,NaN,5.0,458455
top,NaN,NaN,4.0,2000-11-29 20:06:42
freq,NaN,NaN,348971.0,30
first,NaN,NaN,NaN,2000-04-25 23:05:32
last,NaN,NaN,NaN,2003-02-28 17:49:50
mean,3.024512e+03,1.865540e+03,NaN,NaN
std,1.728413e+03,1.096041e+03,NaN,NaN
min,1.000000e+00,1.000000e+00,NaN,NaN
25%,1.506000e+03,1.030000e+03,NaN,NaN


In [86]:
moviesDF_typed.describe(include='all')

,MovieID,Title,Genres
count,3883.000000,3883,3883
unique,NaN,3883,301
top,NaN,"Watcher, The (2000)",Drama
freq,NaN,1,843
mean,1986.049446,NaN,NaN
std,1146.778349,NaN,NaN
min,1.000000,NaN,NaN
25%,982.500000,NaN,NaN
50%,2010.000000,NaN,NaN
75%,2980.500000,NaN,NaN


### verificando duplicatas na base de cadastro

In [87]:
moviesDF_typed.rename(index=str, columns={"MovieID": "id"}, inplace=True)

In [88]:
moviesDF_typed['Title'] = moviesDF_typed['Title'].apply(lambda s: s.strip().upper())

In [89]:
moviesDF_typed.rename(index=str, columns={"Title": "movie_title"}, inplace=True)

In [90]:
movieNamesDistinctCount = moviesDF_typed["movie_title"].nunique()
numMovieIdsDistinctCount = moviesDF_typed["id"].nunique()
print(" Número de distintos na olunas de id de filme e nome de filme são iguais? ?" ,movieNamesDistinctCount == numMovieIdsDistinctCount,movieNamesDistinctCount, numMovieIdsDistinctCount )

 Número de distintos na olunas de id de filme e nome de filme são iguais? ? True 3883 3883


In [91]:
duplicatedFilmsNames = (moviesDF_typed["movie_title"].value_counts() != 1)
duplicatedFilmsNames = duplicatedFilmsNames[duplicatedFilmsNames.values == True].index.tolist()

In [92]:
duplicated_films = moviesDF_typed[moviesDF_typed["movie_title"].isin(duplicatedFilmsNames)]

In [93]:
duplicated_films

,id,movie_title,Genres


### executando o join dos datasets

In [94]:
moviesDF_typed["id"] = moviesDF_typed["id"].astype(int)
ratingsDF_typed["id"] = ratingsDF_typed["id"].astype(int)

In [95]:
# df = pd.merge(ratingsDF_typed, moviesDF_typed, how='inner', on='id')
df = pd.merge(ratingsDF_typed, moviesDF_typed, how='inner', on=['id'])

conferiando se o merge foi executado corretamente

In [96]:
print("Tamanho do dataset após o join em #linhas :", df['id'].count())

Tamanho do dataset após o join em #linhas : 1000209


In [97]:
df.columns

Index(['uid', 'id', 'rating', 'timestamp', 'movie_title', 'Genres'], dtype='object')

A distribuição de IDs de filmes é ainda mais distante da distribuição uniforme. Talvez, filmes com Ids próximos pertençam ao mesmo gênero. Muito provavelmente há generos de filmes mais assistidos.

Segundo os metadados da base, é esperado que cada usuário tenha dado nota para 20 filmes em média. gostaríamos de agrupar os filmes vistos pelos usuáros numa única linha, de forma a criar as "cestas"

In [98]:
numDuplicatedRatings = (df.groupby(['uid', 'id']).agg({"id": len})['id'] != 1).sum()
print("Será que algum usuário deu duas notas par algum filme?", numDuplicatedRatings)

Será que algum usuário deu duas notas par algum filme? 0


In [99]:
numDuplicatedRatings

0

In [100]:
df_dedup = df[df.duplicated(subset=['uid', 'id'], keep='first') == False]

In [101]:
numDuplicatedRatings = (df_dedup.groupby(['uid', 'id']).agg({"id": len})['id'] != 1).sum()
print("Será que algum usuário deu duas notas par algum filme?", numDuplicatedRatings)

Será que algum usuário deu duas notas par algum filme? 0


### Write deduplicated df to file

In [102]:
df_dedup.describe(include='all')

,uid,id,rating,timestamp,movie_title,Genres
count,1.000209e+06,1.000209e+06,1000209.0,1000209,1000209,1000209
unique,NaN,NaN,5.0,458455,3706,301
top,NaN,NaN,4.0,2000-11-29 20:06:42,AMERICAN BEAUTY (1999),Comedy
freq,NaN,NaN,348971.0,30,3428,116883
first,NaN,NaN,NaN,2000-04-25 23:05:32,NaN,NaN
last,NaN,NaN,NaN,2003-02-28 17:49:50,NaN,NaN
mean,3.024512e+03,1.865540e+03,NaN,NaN,NaN,NaN
std,1.728413e+03,1.096041e+03,NaN,NaN,NaN,NaN
min,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN
25%,1.506000e+03,1.030000e+03,NaN,NaN,NaN,NaN


In [103]:
df_dedup.to_pickle("..\\ProcessedData\\df_1m.pkl")